# LAB 4: ANALZING NETWORK DATA LOG

In [118]:
import numpy as np
import pandas as pd
import seaborn as sb
import matplotlib.pyplot as plt 
sb.set() 





In [119]:
pip install ipwhois


In [120]:
from ipwhois import IPWhois

format = ['Type', 'sflow_agent_address', 'inputPort', 'outputPort',
               'src_MAC', 'dst_MAC', 'ethernet_type', 'in_vlan', 'out_vlan',
               'src_IP', 'dst_IP', 'IP_protocol', 'ip_tos', 'ip_ttl',
               'src_transport_port', 'dst_transport_port', 'tcp_flags',
               'packet_size', 'IP_size', 'sampling_rate', '???']

In [121]:
df = pd.read_csv('Data_3.csv', header = None, names = format)
df.drop('???', axis=1, inplace = True) # Drop column with null values
df.head(10)
     

,Type,sflow_agent_address,inputPort,outputPort,src_MAC,dst_MAC,ethernet_type,in_vlan,out_vlan,src_IP,dst_IP,IP_protocol,ip_tos,ip_ttl,src_transport_port,dst_transport_port,tcp_flags,packet_size,IP_size,sampling_rate
0,FLOW,203.30.38.251,137,200,d404ff55fd4d,80711fc76001,0x0800,919,280,130.246.176.22,140.115.32.81,6,0x00,50,51216,23505,0x10,1518,1500,2048
1,FLOW,203.30.38.251,129,193,609c9f851b00,0031466b23cf,0x0800,11,919,155.69.160.32,64.233.188.128,6,0x00,56,23159,80,0x10,74,52,2048
2,FLOW,203.30.38.251,137,200,d404ff55fd4d,80711fc76001,0x0800,919,280,130.246.176.53,140.115.32.83,6,0x00,50,50041,20739,0x10,1518,1500,2048
3,FLOW,203.30.38.251,129,135,609c9f851b00,002688cd5fc7,0x0800,11,919,155.69.160.32,54.169.174.79,17,0x00,120,54241,26510,0x10,116,94,2048
4,FLOW,203.30.38.251,130,199,00239cd087c1,544b8cf9a7df,0x0800,919,600,137.132.228.15,193.62.192.8,6,0x00,56,53923,34262,0x10,70,52,2048
5,FLOW,203.30.38.251,129,135,609c9f851b00,002688cd5fc7,0x0800,11,919,155.69.160.32,54.255.221.151,17,0x00,120,55069,26509,0x10,113,91,2048
6,FLOW,203.30.38.251,130,199,00239cd087c1,544b8cf9a7df,0x0800,919,600,137.132.250.8,193.62.193.9,6,0x00,62,27419,80,0x10,58,40,2048
7,FLOW,203.30.38.251,137,200,d404ff55fd4d,80711fc76001,0x0800,919,280,193.61.196.206,140.110.147.170,6,0x00,240,443,43545,0x18,1518,1500,2048
8,FLOW,203.30.38.251,200,3,80711fc76001,00235ed9b680,0x0800,280,32,137.189.133.62,123.136.64.7,6,0x00,58,6345,34921,0x10,1342,1320,2048
9,FLOW,203.30.38.251,199,130,544b8cf9a7df,00239cd087c1,0x0800,600,919,193.62.192.8,137.132.228.15,6,0x08,55,34262,53923,0x10,1442,1420,2048


## Excercise 4A: Top Talkers and Listerners

### Top Talkers

#### The top_talkers dataframe is created by counting the occurrences of each source IP address (src_IP) and selecting the top 5 with the most packets sent.

In [122]:
def get_organization(ip):
    ipwhois = IPWhois(ip)
    try:
        result = ipwhois.lookup_rdap()
        organization = result.get('network', {}).get('name', 'Unknown')
    except Exception as e:
        organization = 'Unknown'
    return organization

# Get the top 5 talkers
top_talkers = df['src_IP'].value_counts().head(5).reset_index()
top_talkers.columns = ['IP Address', 'no of packets']

# Add organization names for each IP address
top_talkers['organisations'] = top_talkers['IP Address'].apply(get_organization)

print("Top 5 Talkers (Senders):\n", top_talkers)

Top 5 Talkers (Senders):
       IP Address  no of packets     organisations
0   193.62.192.8           3041      EUR-BIO-INST
1  155.69.160.32           2975           NTUNET1
2  130.14.250.11           2604         NLM-ETHER
3  14.139.196.58           2452       NKN-IIT-GUW
4  140.112.8.139           2056  T-NTU.EDU.TW-NET


### Top Listeners

#### The top_listeners dataframe is created by counting the occurrences of each destination IP address (dst_IP) in the dataset and selecting the top 5 with the most packets received.

In [123]:
# Get the top 5 listeners (destination IPs)
top_listeners = df['dst_IP'].value_counts().head(5).reset_index()
top_listeners.columns = ['IP Address', 'no of packets']

# Add organization names for each IP address
top_listeners['organisations'] = top_listeners['IP Address'].apply(get_organization)

print("Top 5 Listeners (Receivers):\n", top_listeners)


Top 5 Listeners (Receivers):
         IP Address  no of packets organisations
0   103.37.198.100           3841  A-STAR-AS-AP
1   137.132.228.15           3715        NUSNET
2   202.21.159.244           2446         RPNET
3  192.101.107.153           2368          PNNL
4     103.21.126.2           2056       IITB-IN


## Excercise 4B: Transport Protocol

#### In this analysis, we are calculating the percentage of TCP and UDP packets in the dataset

In [124]:
# Filter out the relevant rows for TCP (IP_protocol == 6) and UDP (IP_protocol == 17)
tcp_packets = df[df['IP_protocol'] == 6].shape[0]
udp_packets = df[df['IP_protocol'] == 17].shape[0]

# Get the total number of packets
total_packets = df.shape[0]

# Calculate the percentage for TCP and UDP
tcp_percentage = (tcp_packets / total_packets) * 100
udp_percentage = (udp_packets / total_packets) * 100

# Display the results
print(f"TCP Percentage: {tcp_percentage:.2f}%")
print(f"UDP Percentage: {udp_percentage:.2f}%")

# You can also create a summary DataFrame if needed
protocol_percentage = pd.DataFrame({
    'Header value': ['TCP', 'UDP'],
    'Transport layer protocol': ['Transmission Control Protocol', 'User Datagram Protocol'],
    'Percentage': [tcp_percentage, udp_percentage],
    '# of packets': [tcp_packets, udp_packets]
})

print("\nProtocol Percentage Breakdown:\n", protocol_percentage)


TCP Percentage: 80.82%
UDP Percentage: 13.64%

Protocol Percentage Breakdown:
   Header value       Transport layer protocol  Percentage  # of packets
0          TCP  Transmission Control Protocol   80.818798         56064
1          UDP         User Datagram Protocol   13.639902          9462


## Excercise 4C: Application Protocol

#### We first analyze the top 5 most frequently used destination ports in a network traffic dataset and displays them along with the number of packets sent to each port

In [ ]:

# Group by destination port and count the number of packets
port_counts = df['dst_transport_port'].value_counts().head(5).reset_index()
port_counts.columns = ['Destination IP Port Number', 'Number of Packets']



# Display the top 5 most frequent destination ports with the service name
print("Top 5 Most Frequent Destination Ports and their Services:\n", port_counts)


Top 5 Most Frequent Destination Ports and their Services:
    Destination IP Port Number  Number of Packets
0                         443              13423
1                          80               2647
2                       52866               2068
3                       45512               1356
4                       56152               1341


#### Once we figure out the port number, we  map them accordingly. We got the mapping information from https://www.adminsub.net/tcp-udp-port-finder

In [126]:
port_service_mapping = {
    80: 'HTTP',
    443: 'HTTPS',
    52866: 'Dynamic and/or Private Ports',
    45512: 'Unassigned',
    56152: 'Dynamic and/or Private Ports'
}


# Group by destination port and count the number of packets
port_counts = df['dst_transport_port'].value_counts().head(5).reset_index()
port_counts.columns = ['Destination IP Port Number', 'Number of Packets']

# Map the port numbers to services
port_counts['Service'] = port_counts['Destination IP Port Number'].map(port_service_mapping)

# Display the top 5 most frequent destination ports with the service name
print("Top 5 Most Frequent Destination Ports and their Services:\n", port_counts)

Top 5 Most Frequent Destination Ports and their Services:
    Destination IP Port Number  Number of Packets                       Service
0                         443              13423                         HTTPS
1                          80               2647                          HTTP
2                       52866               2068  Dynamic and/or Private Ports
3                       45512               1356                    Unassigned
4                       56152               1341  Dynamic and/or Private Ports


## Excercise 4D: Traffic

#### Here we calculate the total network traffic in megabits (Mb) based on the IP_size column in a DataFrame (df)

In [130]:


total_traffic = sum(df['IP_size'])
# Assuming IP_size is in number of bits
total_traffic_Mb = total_traffic / (8 * pow(2, 20))
print(f"Total Traffic (Mb) = {total_traffic_Mb:.3f} Mb")

Total Traffic (Mb) = 7.722 Mb


## Excercise 4E: Additional Analysis

#### We can identify the top 5 communication pairs (source IP to destination IP) to understand the most active network interactions. This will provide insights into the major communication flows between devices in the network.

In [131]:
# Calculate the top 5 communication pairs using the correct column names
top_communication_pairs = df.groupby(['src_IP', 'dst_IP']).size().reset_index(name='Count')
top_communication_pairs = top_communication_pairs.sort_values(by='Count', ascending=False).head(5)

# Display the top 5 communication pairs
print("Top 5 Communication Pairs:\n", top_communication_pairs)


Top 5 Communication Pairs:
               src_IP           dst_IP  Count
4224    193.62.192.8   137.132.228.15   3041
975    130.14.250.11   103.37.198.100   2599
1507   14.139.196.58  192.101.107.153   2368
1639   140.112.8.139     103.21.126.2   2056
1297  137.132.228.15     193.62.192.8   1910


#### Using the IP_protocol column, we can calculate the percentage of TCP and UDP traffic in the dataset. This helps identify the dominant transport protocols in the traffic.

In [132]:
# Calculate the count of each transport protocol
protocol_counts = df['IP_protocol'].value_counts(normalize=True) * 100

# Print the protocol distribution
print("Protocol Distribution (TCP vs UDP):")
print(protocol_counts)


Protocol Distribution (TCP vs UDP):
IP_protocol
6      80.818798
17     13.639902
50      2.447744
0       1.817789
47      0.947095
41      0.149921
1       0.106674
381     0.064870
58      0.005766
103     0.001442
Name: proportion, dtype: float64
